# pandas中的连接

在数据处理过程中，连接（joining）是一种基本且常见的操作，它允许我们将不同数据集的信息组合在一起，以便进行更加深入的分析。Pandas 提供了多种连接数据的方法，这些方法可以根据不同的业务逻辑和需求来应用。

首先，让我们导入必要的库：

In [4]:
import pandas as pd
import numpy as np

## 关系型连接

### 连接的基本概念

在关系型数据库中，连接是通过键（key）将不同表中的行链接起来。在 Pandas 中，我们可以使用 `merge` 和 `join` 函数来实现类似的操作。这些函数的 `on` 参数用于指定连接的键。

连接的类型通常有以下几种：

- 左连接（left join）：结果集包括左表的所有行，以及右表中与左表键匹配的行。
![Shows the Left join adding matches between the left and right table to the result table](https://dataschool.com/assets/images/how-to-teach-people-sql/leftJoin/leftJoin_1.gif)

- 右连接（right join）：结果集包括右表的所有行，以及左表中与右表键匹配的行。
  
- 内连接（inner join）：结果集仅包括两个表中键匹配的行。
![Gif of how inner join iterates through the tables](https://dataschool.com/assets/images/how-to-teach-people-sql/innerJoin/innerJoin_3.gif)

- 外连接（full outer join）：结果集包括左表和右表中的所有行，不匹配的地方填充 NaN。
![gif showing part one of a full outer join: the left join](https://dataschool.com/assets/images/how-to-teach-people-sql/fullOuter/fullOuter_1.gif)


当键在一个表中不是唯一的时候，连接会产生笛卡尔积，即每个左表中的键与右表中的键匹配的所有可能组合。

### 值连接

在 Pandas 中，`merge` 函数可以根据一个或多个键将行连接起来。下面是一个左连接的例子：

In [40]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'], 'Gender':['F','M']})

In [41]:
df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [42]:
df2

,Name,Gender
0,Si Li,F
1,Wu Wang,M


In [8]:
df1.merge(df2, on='Name', how='left')

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


如果连接的列名不同，可以使用 `left_on` 和 `right_on` 指定：

In [43]:
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'], 'Gender':['F','M']})

In [44]:
df1

,df1_name,Age
0,San Zhang,20
1,Si Li,30


In [45]:
df2

,df2_name,Gender
0,Si Li,F
1,Wu Wang,M


In [47]:
df1.merge(df2, left_on='df1_name', right_on='df2_name', how='left')

,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


当两个表中有重复的列名时，可以通过 `suffixes` 参数来区分：

In [48]:
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})

In [49]:
df1

,Name,Grade
0,San Zhang,70


In [50]:
df2

,Name,Grade
0,San Zhang,80


In [51]:
df1.merge(df2, on='Name', how='left', suffixes=['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


有时，我们需要根据多个列的组合来执行连接，以确保连接的唯一性：

In [52]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})

In [53]:
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [54]:
df2

,Name,Gender,Class
0,San Zhang,F,two
1,San Zhang,M,one


In [55]:
df1.merge(df2, on=['Name', 'Class'], how='left')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


### 索引连接

在 Pandas 中，也可以使用索引作为连接的键。`join` 函数是专门用于索引连接的，它的使用方式更加简洁：

In [56]:
df1 = pd.DataFrame({'Age':[20,30]}, index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']}, index=pd.Series(['Si Li','Wu Wang'],name='Name'))

In [57]:
df1

,Age
Name,
San Zhang,20
Si Li,30


In [58]:
df2

,Gender
Name,
Si Li,F
Wu Wang,M


In [59]:
df1.join(df2, how='left')

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


如果要使用多级索引进行连接，可以这样操作：

In [60]:
df1 = pd.DataFrame({'Age':[20,21]}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['one', 'two']], names=('Name','Class')))
df2 = pd.DataFrame({'Gender':['F', 'M']}, index=pd.MultiIndex.from_arrays([['San Zhang', 'San Zhang'],['two', 'one']], names=('Name','Class')))

In [61]:
df1

Age
Name      Class     
San Zhang one     20
          two     21

In [62]:
df2

Gender
Name      Class       
San Zhang two        F
          one        M

In [63]:
df1.join(df2)

Age Gender
Name      Class            
San Zhang one     20      M
          two     21      F

## 方向连接

### concat

`concat` 函数用于在一个指定的轴上连接多个 Pandas 对象。它不需要基于键来连接，而是简单地将数据集拼接在一起。

例如，纵向合并数据：

In [64]:
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'], 'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Wu Wang'], 'Age':[40]})

In [65]:
df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [66]:
df2

,Name,Age
0,Wu Wang,40


In [67]:
pd.concat([df1, df2])

,Name,Age
0,San Zhang,20
1,Si Li,30
0,Wu Wang,40


横向合并数据：

In [68]:
df2 = pd.DataFrame({'Grade':[80, 90]})
df3 = pd.DataFrame({'Gender':['M', 'F']})

In [69]:
df2

,Grade
0,80
1,90


In [70]:
df3

,Gender
0,M
1,F


In [71]:
pd.concat([df1, df2, df3], axis=1)

,Name,Age,Grade,Gender
0,San Zhang,20,80,M
1,Si Li,30,90,F


### 序列与表的合并

`append` 方法允许我们将序列添加到 DataFrame 的末尾：

In [73]:
df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [72]:
s = pd.Series(['Wu Wang', 21], index=df1.columns)
df1.append(s, ignore_index=True)

C:\Users\wk\AppData\Local\Temp\ipykernel_992\4269144304.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(s, ignore_index=True)


,Name,Age
0,San Zhang,20
1,Si Li,30
2,Wu Wang,21
